In [4]:
import pandas as pd

default_rank_dir = "/dccstor/colbert-ir/bsiyer/vectordb/indexes/search_dpr_benchmark_1k_q_squad_xor"
default_rank_dir = "/dccstor/colbert-ir/bsiyer/vectordb/weaviate/benchmark_1k_q_PassageAllMiniLML6v2"

info_df = pd.read_json(default_rank_dir + "/ranked_passages.tsv.annotated.metrics")

In [5]:
qas_df = pd.read_json(info_df['arguments']['qas'], lines=True)
ranking_df = pd.read_csv(info_df['arguments']['ranking'], delimiter="\t", names=['qid','pid','rank','score'])
collection_df = pd.read_csv(info_df['arguments']['collection'], delimiter="\t")

In [37]:
ranking_df_100 = ranking_df[ranking_df['rank']<=100]
ranking_df_100 = ranking_df_100.sort_values(by=['qid','rank'])
ranking_df_100.to_csv('/dccstor/srosent3/reranking/weaviate/100test/ranked_passages.tsv', sep='\t', index=False, header=False)

In [3]:
qas_df.sample(5)

,qid,question,answers
419,5522,according to it 's long running tourism slogan...,[Lovers]
245,7965,who wrote the song one is the loneliest number,[Harry Nilsson]
974,7409,how many games did france play in world cup,[6]
752,6470,who has the most world series wins in mlb history,[New York Yankees]
903,2278,when did it become mandatory to have a social ...,[November 1935]


In [4]:
ranking_df.sample(5)

,qid,pid,rank,score
995561,7268,4718,562,0.819237
624236,3297,9817,237,0.782330
178071,4917,3553,72,0.676784
475490,3208,8257,491,0.809845
363922,3766,7005,923,0.873788


In [5]:
collection_df.sample(5)

,id,text,title
570,571,"and applied geometry, especially in his invest...",Philosophy of science
9509,9510,precedence. The current CAG of India is Rajiv ...,Comptroller and Auditor General of India
1339,1340,the 29th Canadian Ministry. Key: Minister of E...,Minister of Environment and Climate Change (Ca...
501,502,"Saturday Night Live cast members , the late-ni...",Saturday Night Live cast members
10530,10531,"Shipman's house in Essex, where Nessa has some...",Gavin & Stacey


In [54]:
# load model from model hub
from primeqa.components.reader.extractive import ExtractiveReader

reader = ExtractiveReader(model="PrimeQA/tydiqa-primary-task-xlm-roberta-large", max_num_answers=10)
reader.load()

{"time":"2023-06-02 15:46:04,659", "name": "ExtractiveQAHead", "level": "INFO", "message": "Loading dropout value 0.1 from config attribute 'hidden_dropout_prob'"}
{"time":"2023-06-02 15:46:05,333", "name": "XLMRobertaModelForDownstreamTasks", "level": "INFO", "message": "Setting task head for first time to 'None'"}


In [3]:

# list of questions and list of list of contexts (list for each question)
def run(questions, contexts, example_ids, passage_ids):
    reader_answers = reader.predict(
        questions, contexts, example_ids=example_ids
    )
    result = {}
    count = 0
    for i, answers_i in reader_answers.items():
        i_result = {}
        for answer in answers_i:
            answer['passage_index'] = passage_ids[count]
        i_result["answers"] = answers_i
        i_result["passages"] = contexts[int(i[i.index('.')+1:])]
        result[i] = i_result
        count += 1
    return result

In [7]:
def get_passages(qas, collection, ranking, num_passages=100):
    questions = []
    contexts = []
    example_ids = []
    pids = []
    for i, row in qas.iterrows():
        q_passages = ranking[ranking['qid'] == row['qid']][:num_passages]
        
        passages = []
        count = 0
        for j, passage in q_passages.iterrows():
            questions.append(row['question'])
            contexts.append([collection.iloc[int(passage['pid'])-1]['title'] + "\n" + collection.iloc[int(passage['pid'])-1]['text']])
            pids.append(str(int(passage['pid'])))
            example_ids.append(f'{row["qid"]}.{count}')
            count+= 1
        # questions.append(row['question'])
        # contexts.append(passages)
    return questions, contexts, example_ids, pids

q, c, e, p = get_passages(qas_df, collection_df, ranking_df)

In [58]:
result = run(q,c,e,p)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Running tokenizer on eval dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

{"time":"2023-06-02 16:23:07,805", "name": "primeqa.mrc.trainers.mrc", "level": "INFO", "message": "The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaModelForDownstreamTasks.forward` and have been ignored: context_idx, offset_mapping, example_idx, example_id."}


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


100%|██████████| 200/200 [00:00<00:00, 258.43it/s]


In [59]:
print(result)

{'6825.0': {'answers': [{'example_id': '6825.0', 'passage_index': '3', 'span_answer_text': 'June 6 to October 3, 1999.', 'span_answer': {'start_position': 384, 'end_position': 410}, 'span_answer_score': -2.6945436894893646, 'confidence_score': 0.16821444440386552}, {'example_id': '6825.0', 'passage_index': '3', 'span_answer_text': '1999.', 'span_answer': {'start_position': 405, 'end_position': 410}, 'span_answer_score': -2.83103010058403, 'confidence_score': 0.14675333866057014}, {'example_id': '6825.0', 'passage_index': '3', 'span_answer_text': 'October 3, 1999.', 'span_answer': {'start_position': 394, 'end_position': 410}, 'span_answer_score': -3.0288747251033783, 'confidence_score': 0.1204107223049372}, {'example_id': '6825.0', 'passage_index': '3', 'span_answer_text': 'from June 6 to October 3, 1999.', 'span_answer': {'start_position': 379, 'end_position': 410}, 'span_answer_score': -3.280783623456955, 'confidence_score': 0.09359712678255057}, {'example_id': '6825.0', 'passage_inde

In [1]:
# load result
from glob import glob
import json

files = glob("/dccstor/srosent3/reranking/weaviate/reader_answers*.json")
files.sort()
print(files)

results = {}
for file_name in files:
    with open(file_name) as json_file:
        data = json.load(json_file)
        results.update(data)
print(len(results))

['/dccstor/srosent3/reranking/weaviate/reader_answers_0-200.json', '/dccstor/srosent3/reranking/weaviate/reader_answers_200-400.json', '/dccstor/srosent3/reranking/weaviate/reader_answers_400-600.json', '/dccstor/srosent3/reranking/weaviate/reader_answers_600-800.json', '/dccstor/srosent3/reranking/weaviate/reader_answers_800-1000.json']
100000


In [8]:
count = 0
for result in results:
    results[result]['passage_score'] = ranking_df.iloc[count]['score']
    for answer in results[result]['answers']:
        answer['passage_index'] = p[count]
    count += 1


results    

KeyboardInterrupt: 

In [5]:
total_confidence = 0
for answer in results['166.1']['answers']:
    print(answer['confidence_score'])
    total_confidence += answer['confidence_score']
total_confidence

0.25970915172078096
0.10473397938267179
0.09725159809099625
0.09474635771790132
0.09405820137165674
0.08174373425206458
0.07875713855902512
0.06469647607349042
0.06311307074650431
0.061190292084908574


1.0

In [24]:
def update(results):
    min_p = 1000
    max_p = 0
     
    for r in results:
        example_id, original_rank = results[r]['answers'][0]['example_id'].split('.')
        results[r]['answers'][0]['example_id'] = example_id
        results[r]['answers'][0]['original_rank'] = original_rank
        results[r]['answers'][0]['passage_score'] = results[r]['passage_score']
               
        for r in results:
            if results[r][0]['passage_score'] < min_p:
                min_p = results[r]['passage_score']
            if results[r][0]['passage_score'] > max_p:
                max_p = results[r]['passage_score']

        # normalize scores of first answer
        if original_rank == 100:
                min_p = 1000
                max_p = 0



# sort by confidence score
def rank_by_confidence(results, alpha=.7, beta=.3):
    answer_per_passage = []
    for r in results:
        answer_per_passage.append(results[r]['answers'][0])
    sorted_answers = sorted(answer_per_passage, key=lambda x:(x['example_id'],alpha*x['passage_score']+beta*x['confidence_score']), reverse=True) 
    return sorted_answers

# count how many different and unique answers per reader query
def rank_by_count(results):
    answer_per_passage = []
    for r in results:
        start_offsets = []
        end_offsets = []
        for answer in results[r]['answers']:
            # answer['example_id'] = answer['example_id'][:answer['example_id'].index('.')]
            start_offsets.append(answer['span_answer']['start_position'])
            end_offsets.append(answer['span_answer']['end_position'])
        start_offsets.sort()
        end_offsets.sort()

        cur = -1
        count = 0
        for i in range(len(start_offsets)):
            if start_offsets[i] > cur:
                count += 1
                cur = end_offsets[i]
        results[r]['answers'][0]['count'] = count
        answer_per_passage.append(results[r]['answers'][0])
    sorted_answers = sorted(answer_per_passage, key=lambda x:(x['example_id'],x['count'],x['original_rank']), reverse=True) 
    return sorted_answers

# vote across all retrieved answers (first answer found)
def rank_by_vote(results):
    answers = {}
    answer_per_passage = []

    for r in results:
        example_id = results[r]['answers'][0]['example_id']
        if example_id not in answers:
            answers[example_id] = {}
        if results[r]['answers'][0]['span_answer_text'] in answers[example_id]:
            answers[example_id][results[r]['answers'][0]['span_answer_text']] += 1
        else:
            answers[example_id][results[r]['answers'][0]['span_answer_text']] = 1
        answer_per_passage.append(results[r]['answers'][0])
    for answer in answer_per_passage:
        answer['count'] = answers[answer['example_id']][answer['span_answer_text']]
    sorted_answers = sorted(answer_per_passage, key=lambda x:(x['example_id'],x['count'],x['original_rank']), reverse=True) 
    return sorted_answers
    

# update(results)
alpha_thres = 0.0
beta_thres = 1.0
new_rank_result = rank_by_confidence(results, alpha=alpha_thres, beta=beta_thres)

In [28]:
new_rank_result

[{'example_id': '997',
  'passage_index': '629',
  'span_answer_text': 'Patricia Richardson',
  'span_answer': {'start_position': 424, 'end_position': 443},
  'span_answer_score': 6.501530647277832,
  'confidence_score': 0.8161611187761691,
  'original_rank': '3',
  'passage_score': 82.61978149414062},
 {'example_id': '997',
  'passage_index': '2691',
  'span_answer_text': 'Ross Bagley',
  'span_answer': {'start_position': 410, 'end_position': 421},
  'span_answer_score': -2.9185858368873596,
  'confidence_score': 0.7157120000029131,
  'original_rank': '7',
  'passage_score': 82.60868072509766},
 {'example_id': '997',
  'passage_index': '635',
  'span_answer_text': 'Jonathan Taylor Thomas',
  'span_answer': {'start_position': 585, 'end_position': 607},
  'span_answer_score': -0.4141874313354492,
  'confidence_score': 0.6428726875710338,
  'original_rank': '1',
  'passage_score': 82.62447357177734},
 {'example_id': '997',
  'passage_index': '630',
  'span_answer_text': 'Patricia Richard

In [25]:
def print_results(results):
    output = []
    index = 1
    for result in results:
        output.append([result['example_id'], result['passage_index'], index, result['confidence_score']])
        index+=1
        if index == 101:
            index = 1
    return output

result_df = pd.DataFrame(print_results(new_rank_result),columns=['qid','pid','rank','score'])

In [23]:
result_df.sample(5)

,qid,pid,rank,score
72169,300,3607,70,0.371199
91284,1697,5924,85,0.130484
27735,6571,3847,36,0.128751
31390,6302,3514,91,0.676402
42277,5422,46,78,0.687540


In [26]:
# result_1k_df = pd.concat([result_df,ranking_df[ranking_df['rank']>100]])

In [27]:
# result_1k_df['qid'] = result_1k_df['qid'].astype(str)
# len(result_1k_df)


1000000

In [28]:
# result_1k_df = result_1k_df.sort_values(by=['qid','rank'], ascending=True)

In [26]:
print('/dccstor/srosent3/reranking/weaviate/confidence/confidence_ps.' + str(alpha_thres) + '.' + str(beta_thres))
result_df.to_csv('/dccstor/srosent3/reranking/weaviate/confidence/confidence_ps.' + str(alpha_thres) + '.' + str(beta_thres) + '/ranked_passages.tsv', sep='\t', index=False, header=False)

/dccstor/srosent3/reranking/weaviate/confidence/confidence_ps.0.0.1.0


In [28]:
# 1. Let's compute some stats of how many times the answers occur across passages and a single passage.
# 2. Next: do some manual analysis to see if the answer in the passage is actually the answer. 
# Match Per Single Passage Count: 0 count: 273 1 count: 598 2 count: 89 3 and up: 40 (1000 total, 87% 1 or less occurrences)
for i, question in qas_df.iterrows():
    # print(question)
    count = 0
    multiple_count = 0
    max_count = 0
    max_rank = 0
    result_q_df = result_df[result_df['qid']==str(question['qid'])]
    first_index = -1
    max_first_count = 0
    index = 0
    for j, answer in result_q_df.iterrows():
        passage = collection_df[collection_df['id']==int(answer['pid'])]

        for answer_text in question['answers']:
            if answer_text.lower() in passage.iloc[0]['text'].lower():
                count += 1 
                count_multiple_passage = passage.iloc[0]['text'].lower().count(answer_text.lower())
                if first_index == -1:
                    first_index = index
                    max_first_count = count_multiple_passage
                if count_multiple_passage > max_count:
                    max_count = count_multiple_passage
                    max_rank = index
                multiple_count += count_multiple_passage
        index+=1
        if count == 0:
            multiple = 0
        else:
            multiple = multiple_count/count
        # print(passage)
    print(f'{question["question"]}\t{question["answers"]}\t{count}\t{multiple}\t{max_count}\t{max_rank}\t{max_first_count}\t{first_index}')

how many episodes in season 2 sex and the city	['18']	20	1.65	4	36	1	1
the part of peripheral nerve tissue that forms the myelin sheath is the	['Schwann cells']	1	1.0	1	34	1	34
where is the continental divide on i 70	['The Eisenhower Tunnel']	1	1.0	1	2	1	2
when was the first national draft used in the united states	['American Civil War']	5	1.0	1	2	1	2
who has the record for most super bowl losses	['Patriots', 'Denver Broncos']	37	1.7837837837837838	6	35	2	5
who are going to be the judges on america 's got talent 2017	['Howie Mandel', 'Heidi Klum', 'Simon Cowell', 'Mel B']	11	1.0	1	1	1	1
what year was pride and prejudice set in	['c. 1812']	0	0	0	0	0	-1
how many episodes in love hate season 1	['4']	46	1.5434782608695652	5	41	2	3
who is the singer of mumford and sons married to	['Carey Mulligan']	0	0	0	0	0	-1
when does drag race all stars 3 premiere	['January 25 , 2018']	0	0	0	0	0	-1
when did kareem abdul jabbar reach 30000 points	['1983 -- 84']	0	0	0	0	0	-1
where does the name du plessis

In [38]:
results_df = pd.DataFrame.from_dict(results)

# new_rank_result_df[new_rank_result_df['example_id']=='166']

In [40]:
results_df.sample(1)

,6825.0,6825.1,6825.2,6825.3,6825.4,6825.5,6825.6,6825.7,6825.8,6825.9,...,6716.90,6716.91,6716.92,6716.93,6716.94,6716.95,6716.96,6716.97,6716.98,6716.99
passage_score,0.279068,0.289527,0.33683,0.431202,0.449478,0.478856,0.520721,0.521583,0.530632,0.559315,...,0.862017,0.86206,0.862098,0.862184,0.86219,0.862214,0.862248,0.862314,0.862318,0.862325


In [1]:
# normalize passage scores
import pandas as pd

doc_scores_file = "/dccstor/srosent3/reranking/primeqa-dpr/ranked_passages.tsv"

doc_scores = pd.read_csv(doc_scores_file, delimiter='\t', names=['qid','pid','rank','score'])


In [2]:

from tqdm import tqdm 

qids = doc_scores['qid'].unique()

normalized_df = pd.DataFrame(columns=['qid','pid','rank','score'])


for qid in tqdm(qids):
    docs = doc_scores[doc_scores['qid'] == qid]
    max_score = docs.iloc[0]['score']
    min_score = docs.iloc[-1]['score']

    for i, row in docs.iterrows():
        docs['score'].at[i] = (row['score'] - min_score) / (max_score - min_score)
    normalized_df = pd.concat([normalized_df, docs], ignore_index=True)


  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_1339389/3777439022.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs['score'].at[i] = (row['score'] - min_score) / (max_score - min_score)
100%|██████████| 1000/1000 [03:10<00:00,  5.25it/s]


In [3]:
normalized_df[normalized_df['qid'] == 6825]

,qid,pid,rank,score
0,6825,7533,0,1.000000e+00
1,6825,7524,1,8.936167e-01
2,6825,1,2,8.785301e-01
3,6825,7526,3,8.359135e-01
4,6825,7523,4,7.946441e-01
...,...,...,...,...
995,6825,8164,995,1.577161e-04
996,6825,8467,996,7.648282e-05
997,6825,2915,997,6.935709e-05
998,6825,9309,998,4.750485e-07


In [ ]:
normalized_df.to_csv('/dccstor/srosent3/reranking/primeqa-dpr/ranked_passages_normalized.tsv', sep='\t')

In [1]:
# load result
from glob import glob
import json

files = glob("/dccstor/srosent3/reranking/debug-sap/reader_answers*.json")
files.sort()
print(files)

results = {}
for file_name in files:
    with open(file_name) as json_file:
        data = json.load(json_file)
        results.update(data)
print(len(results))

['/dccstor/srosent3/reranking/debug-sap/reader_answers_0-2.json']
2


In [2]:
results['0']

{'answers': [{'example_id': '0',
   'passage_index': '238e750c2cee4afeb07bd9514807c0d3',
   'span_answer_text': 'add master data items from the central procurement hub to external systems, both parent and child, as event content',
   'span_answer': {'start_position': 519, 'end_position': 634},
   'span_answer_score': 10.102726072072983,
   'confidence_score': 0.030909752017710374},
  {'example_id': '0',
   'passage_index': 'f1ed148b14014e5cb68419948788a487',
   'span_answer_text': 'After the central contract is created and approved, outline agreements such as contracts or scheduling agreements are created based on this central contract in the connected systems, where they can be used as sources of supply. Note that you can access central contracts only if you have authorization for the respective central purchasing group, central purchasing organization, and document type. You can use this app only if your configuration expert has configured the Central Purchase Contracts scenario in C

In [3]:
# sort by confidence score
def rank_by_confidence(results, alpha=.7, beta=.3):
    answer_per_passage = []
    for qid in results:
        for answer in results[qid]['answers']:
            answer_per_passage.append(answer)
    sorted_answers = sorted(answer_per_passage, key=lambda x:(x['example_id'],alpha*x['passage_score']+beta*x['confidence_score']), reverse=True) 
    return sorted_answers

rank_by_confidence(results)

KeyError: 'passage_score'